# Cargar y probar un modelo de Keras

## Cargar modelo desde .h5

El modelo se puede cargar desde un archivo .json. Al ejecutar el siguiente cuadro de código podremos cargar uno desde el explorador de archivos.

In [14]:
from ipywidgets import FileUpload, Output
from tensorflow.keras import Model, models
import json
import os
from io import StringIO

if 'loaded_model' not in globals():
    global loaded_model
    loaded_model = None

def mostrar_h5_upload():
    h5_upload = FileUpload(accept= '.h5', 
    multiple=False)
    display(h5_upload)
    h5_upload.observe(handle_h5_upload, names='value')

def handle_h5_upload(change):
    *_, (_, f) = change['new'].items()
    h5_content = f['content']
    h5_uploaded_filepath = "kerasModelUpload.h5"
    with open(h5_uploaded_filepath,"w+b") as h5_file:
        h5_file.write(h5_content)
    global loaded_model
    loaded_model = models.load_model(h5_uploaded_filepath)
    print("Modelo cargado correctamente")
    print("Ejecuta el siguiente cuadro de código para verlo...")
    os.remove(h5_uploaded_filepath)
        
if(loaded_model is not None):
    answer = input("Ya hay un modelo cargado ¿Quieres borrarlo? [s/n]: ")
    if answer == "s":
        loaded_model = None
        print("Se ha borrado el modelo que se había cargado")
        mostrar_h5_upload()
    elif answer == "n":
        print("No se ha borrado el modelo.")
    else:
        print("No se ha borrado el modelo.")
else:
    loaded_model=None
    mostrar_h5_upload()

Ya hay un modelo cargado ¿Quieres borrarlo? [s/n]: s
Se ha borrado el modelo que se había cargado


FileUpload(value={}, accept='.h5', description='Upload')

Modelo cargado correctamente
Ejecuta el siguiente cuadro de código para verlo...
Modelo cargado correctamente
Ejecuta el siguiente cuadro de código para verlo...


In [15]:
loaded_model.summary()

Model: "sequential_63"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_188 (Conv2D)          (None, 126, 126, 4)       40        
_________________________________________________________________
max_pooling2d_188 (MaxPoolin (None, 42, 42, 4)         0         
_________________________________________________________________
conv2d_189 (Conv2D)          (None, 38, 38, 8)         808       
_________________________________________________________________
max_pooling2d_189 (MaxPoolin (None, 12, 12, 8)         0         
_________________________________________________________________
conv2d_190 (Conv2D)          (None, 6, 6, 16)          6288      
_________________________________________________________________
max_pooling2d_190 (MaxPoolin (None, 3, 3, 16)          0         
_________________________________________________________________
flatten_63 (Flatten)         (None, 144)             

Tambien podemos cargar el modelo manualmente si sabemos el filepath del fichero .h5.

In [6]:
h5_filepath = "carlosNet-aug-final.h5" # Sustituir por el filepath del archivo a cargar
loaded_model = models.load_model(h5_filepath)
loaded_model.summary()

Model: "sequential_63"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_188 (Conv2D)          (None, 126, 126, 4)       40        
_________________________________________________________________
max_pooling2d_188 (MaxPoolin (None, 42, 42, 4)         0         
_________________________________________________________________
conv2d_189 (Conv2D)          (None, 38, 38, 8)         808       
_________________________________________________________________
max_pooling2d_189 (MaxPoolin (None, 12, 12, 8)         0         
_________________________________________________________________
conv2d_190 (Conv2D)          (None, 6, 6, 16)          6288      
_________________________________________________________________
max_pooling2d_190 (MaxPoolin (None, 3, 3, 16)          0         
_________________________________________________________________
flatten_63 (Flatten)         (None, 144)             

## Probando el modelo

### Cargar los datos

In [16]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

simple_datagen = ImageDataGenerator(rescale = 1.0/255.)

dataset_folder="dataset"
batch_size=30

train_dir=f"{dataset_folder}/train"
test_dir=f"{dataset_folder}/test"
val_dir=f"{dataset_folder}/val"

train_set = simple_datagen.flow_from_directory(train_dir,
                  batch_size=batch_size,
                  class_mode='categorical',
                  target_size=(128, 128),
                  color_mode="grayscale")

test_set = simple_datagen.flow_from_directory(test_dir,
                  batch_size=batch_size,
                  class_mode = 'categorical',
                  target_size = (128, 128),
                  color_mode="grayscale")

val_set = simple_datagen.flow_from_directory(val_dir,
                  batch_size=batch_size,
                  class_mode = 'categorical',
                  target_size = (128, 128),
                  color_mode="grayscale")

Found 14400 images belonging to 6 classes.
Found 3600 images belonging to 6 classes.
Found 3600 images belonging to 6 classes.


### Evaluar

In [18]:
metrics_test = loaded_model.evaluate(
             test_set,
             batch_size=30,
             steps= test_set.n // 30,
             verbose=1,
             return_dict=True)

acc_test    = metrics_test['accuracy']
loss_test   = metrics_test['loss']

print(f"Precisión para el conjunto de Prueba: {acc_test}")
print(f"Pérdida para el conjunto de Prueba: {loss_test}")

120/120 [==============================] - 2s 18ms/step - loss: 0.0029 - accuracy: 0.9997
Precisión para el conjunto de Prueba: 0.9997222423553467
Pérdida para el conjunto de Prueba: 0.0029315564315766096
